In [1]:
import pandas as pd
import numpy as np
from pyspark.sql.functions import monotonically_increasing_id,min,avg
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator

### Loading the Dataset

In [2]:
df=spark.read.csv("/FileStore/tables/merged.csv", header="true", inferSchema="true")

In [3]:
df.show()

+---+--------------------+--------------------+------+
_c0| user_id| business_id|rating|
+---+--------------------+--------------------+------+
 0|zvNimI98mrmhgNOOr...|JwUE5GmEO-sH1FuwJ...| 4.0|
 1|p4ySEi8PEli0auZGB...|JwUE5GmEO-sH1FuwJ...| 4.0|
 2|ZYaumz29bl9qHpu-K...|JwUE5GmEO-sH1FuwJ...| 4.0|
 3|SvS7NXWG2B2kFoaHa...|JwUE5GmEO-sH1FuwJ...| 4.0|
 4|qOYI9O0ecMJ9VaqcM...|JwUE5GmEO-sH1FuwJ...| 4.0|
 5|EEYwj6_t1OT5WQGyp...|JwUE5GmEO-sH1FuwJ...| 4.0|
 6|bfww9ItvUBeRsFQ4U...|JwUE5GmEO-sH1FuwJ...| 4.0|
 7|Au3Qs-AAZEWu2_4gI...|JwUE5GmEO-sH1FuwJ...| 4.0|
 8|MnXcXwr0keJpkIiwu...|JwUE5GmEO-sH1FuwJ...| 4.0|
 9| #NAME?|JwUE5GmEO-sH1FuwJ...| 4.0|
 10|MPKMK36Xp0GuaWaXl...|JwUE5GmEO-sH1FuwJ...| 4.0|
 11|pgKOL4k5tWCWS9VIc...|JwUE5GmEO-sH1FuwJ...| 4.0|
 12|4K7YXIl77EOHlTfSU...|JwUE5GmEO-sH1FuwJ...| 4.0|
 13|MDiCwtVo-Aq9cD5bA...|JwUE5GmEO-sH1FuwJ...| 4.0|
 14|wC8r-m6KHifL6R2i8...|JwUE5GmEO-sH1FuwJ...| 4.0|
 15|4ih20jNIEPIkhkh-u...|JwUE5GmEO-sH1FuwJ...| 4.0|
 16|Bld8sdbL48rJzVXo4...|JwUE5GmEO-sH1FuwJ...| 4.0|
 17|OtpHvQFGbZ4sONaQA...|JwUE5GmEO-sH1FuwJ...| 4.0|
 18|nJ_6gzvBj0mF943eW...|JwUE5GmEO-sH1FuwJ...| 4.0|
 19|RvweNJFVkR3ttkWsI...|JwUE5GmEO-sH1FuwJ...| 4.0|
+---+--------------------+--------------------+------+
only showing top 20 rows

In [4]:
users=df.select("user_id").distinct().coalesce(1)

## Creating Numeric Id for users

In [5]:

users=users.withColumn("UserId",monotonically_increasing_id()).persist()
users.show()

+--------------------+------+
 user_id|UserId|
+--------------------+------+
I7dYsjvl3b66gL4Kh...| 0|
RBhNiqaAU95xXYRf4...| 1|
vU4N-TaRwk5GtAc97...| 2|
hh6u9Ny2XY0TiWLha...| 3|
m3AVaQrW0aLEgg6-Y...| 4|
a0PfJ18hkhemRlEuM...| 5|
8KiaJ22iKNFVgWCc6...| 6|
4B5vxhyiEOIxaGF-J...| 7|
CC4g_0ZVz3P_0He6p...| 8|
Tg6Z_SL53-c-nyGWt...| 9|
wnsWrAyyn2SEGgVVx...| 10|
iXH-s7352a9BgJF3M...| 11|
CZndIr_tyqmoly0xm...| 12|
WG6fad_27Fm7GAiH2...| 13|
yfQwWLYtqwHKiky1h...| 14|
Up3fKmpiKHSx6uGA1...| 15|
v686jxS1tqbknbANW...| 16|
_A7k2p_9zkFy3y8VB...| 17|
j3pNYfD2jIU2JEQ64...| 18|
6ThmkFMpHzdCfxkRP...| 19|
+--------------------+------+
only showing top 20 rows

In [6]:
restaurants=df.select("business_id").distinct().coalesce(1)

## Creating Numeric Id for restaurants

In [7]:

restaurants=restaurants.withColumn("restaurantId",monotonically_increasing_id()).persist()
restaurants.show()

+--------------------+------------+
 business_id|restaurantId|
+--------------------+------------+
89O2bCyo92IpEEvgF...| 0|
vGCetjJwnPU5bqoNx...| 1|
w7XfqsXO3ZEtoZs8_...| 2|
0p3MbCriuvH9hgqRK...| 3|
8GlGakyGqp9WvToKc...| 4|
9Tjj3Zg-PPFCIUB_I...| 5|
xHJPgPO3wF-F3Ep8V...| 6|
dylp6k9_Du-oojW5l...| 7|
K2BxGfj37me-kf__e...| 8|
Kr7Vx-Yk7BpKcbGH6...| 9|
iun1duUP2DH4i05GF...| 10|
Bt2JuKNLqSnVympH-...| 11|
O_Pu4mKGOqQVlmH07...| 12|
wS6Qh5NkoPKDwjA_v...| 13|
v5mjUyUZeWfLTdxXf...| 14|
vNSSn98ePYBbCwRCD...| 15|
O5lo-GKCG3cX8X4bh...| 16|
TWtdZw-QCgG71af01...| 17|
lN-5-YTsaJr_IByyA...| 18|
wXri8QF92KOYe2INy...| 19|
+--------------------+------------+
only showing top 20 rows

In [8]:
restaurant_rating=df.join(users,"user_id","left").join(restaurants,"business_id","left")

In [9]:
restaurant_rating.show()

+--------------------+--------------------+---+------+------+------------+
 business_id| user_id|_c0|rating|UserId|restaurantId|
+--------------------+--------------------+---+------+------+------------+
JwUE5GmEO-sH1FuwJ...|zvNimI98mrmhgNOOr...| 0| 4.0| 5066| 171|
JwUE5GmEO-sH1FuwJ...|p4ySEi8PEli0auZGB...| 1| 4.0| 6900| 171|
JwUE5GmEO-sH1FuwJ...|ZYaumz29bl9qHpu-K...| 2| 4.0| 6009| 171|
JwUE5GmEO-sH1FuwJ...|SvS7NXWG2B2kFoaHa...| 3| 4.0| 2737| 171|
JwUE5GmEO-sH1FuwJ...|qOYI9O0ecMJ9VaqcM...| 4| 4.0| 1876| 171|
JwUE5GmEO-sH1FuwJ...|EEYwj6_t1OT5WQGyp...| 5| 4.0| 4145| 171|
JwUE5GmEO-sH1FuwJ...|bfww9ItvUBeRsFQ4U...| 6| 4.0| 4295| 171|
JwUE5GmEO-sH1FuwJ...|Au3Qs-AAZEWu2_4gI...| 7| 4.0| 6901| 171|
JwUE5GmEO-sH1FuwJ...|MnXcXwr0keJpkIiwu...| 8| 4.0| 4699| 171|
JwUE5GmEO-sH1FuwJ...| #NAME?| 9| 4.0| 637| 171|
JwUE5GmEO-sH1FuwJ...|MPKMK36Xp0GuaWaXl...| 10| 4.0| 3720| 171|
JwUE5GmEO-sH1FuwJ...|pgKOL4k5tWCWS9VIc...| 11| 4.0| 5585| 171|
JwUE5GmEO-sH1FuwJ...|4K7YXIl77EOHlTfSU...| 12| 4.0| 1990| 171|
JwUE5GmEO-sH1FuwJ...|MDiCwtVo-Aq9cD5bA...| 13| 4.0| 4998| 171|
JwUE5GmEO-sH1FuwJ...|wC8r-m6KHifL6R2i8...| 14| 4.0| 5483| 171|
JwUE5GmEO-sH1FuwJ...|4ih20jNIEPIkhkh-u...| 15| 4.0| 4764| 171|
JwUE5GmEO-sH1FuwJ...|Bld8sdbL48rJzVXo4...| 16| 4.0| 7105| 171|
JwUE5GmEO-sH1FuwJ...|OtpHvQFGbZ4sONaQA...| 17| 4.0| 4028| 171|
JwUE5GmEO-sH1FuwJ...|nJ_6gzvBj0mF943eW...| 18| 4.0| 2591| 171|
JwUE5GmEO-sH1FuwJ...|RvweNJFVkR3ttkWsI...| 19| 4.0| 6796| 171|
+--------------------+--------------------+---+------+------+------------+
only showing top 20 rows

In [10]:
restaurant_rating=restaurant_rating.select("UserId","restaurantId","rating")

## Checking the Sparsity of the DataFrame

In [11]:

# Count the total number of ratings in the dataset
numerator = restaurant_rating.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = restaurant_rating.select("userId").distinct().count()
num_restaurants = restaurant_rating.select("restaurantId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_restaurants

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is 99.56% empty.

### Descriptive Analysis

In [12]:
# Min num ratings for restaurants
print("Restaurants with the fewest ratings: ")
restaurant_rating.groupBy("restaurantId").count().select(min("count")).show()

#Avg num ratings per restaurants
print("Avg num ratings per restaurants: ")
restaurant_rating.groupBy("restaurantId").count().select(avg("count")).show()

# Min num ratings for user
print("User with the fewest ratings: ")
restaurant_rating.groupBy("userId").count().select(min("count")).show()

# Avg num ratings per users
print("Avg num ratings per user: ")
restaurant_rating.groupBy("userId").count().select(avg("count")).show()

Restaurants with the fewest ratings: 
+----------+
min(count)|
+----------+
 1|
+----------+

Avg num ratings per restaurants: 
+------------------+
 avg(count)|
+------------------+
32.379120879120876|
+------------------+

User with the fewest ratings: 
+----------+
min(count)|
+----------+
 1|
+----------+

Avg num ratings per user: 
+------------------+
 avg(count)|
+------------------+
3.2255062944718116|
+------------------+

### Fitting the ALS Model

In [13]:
(training_data,test_data)=restaurant_rating.randomSplit([0.8,0.2],seed=101)

In [14]:
als=ALS(userCol="UserId",itemCol="restaurantId",ratingCol="rating",rank=25,maxIter=20,regParam=.1,coldStartStrategy="drop",nonnegative=True,implicitPrefs=False)

In [15]:
model=als.fit(training_data)

In [16]:
test_predictions = model.transform(test_data)
test_predictions.show()

+------+------------+------+----------+
UserId|restaurantId|rating|prediction|
+------+------------+------+----------+
 2858| 148| 4.0| 3.8692846|
 6439| 148| 4.0| 3.8843544|
 6250| 148| 4.0| 3.8937685|
 5117| 471| 3.5| 3.386865|
 1766| 471| 3.5| 3.4232032|
 4028| 471| 3.5| 3.4027534|
 5947| 471| 3.5| 3.3927274|
 839| 471| 3.5| 3.4315343|
 4765| 471| 3.5| 3.38546|
 6381| 471| 3.5| 3.430068|
 6285| 471| 3.5| 3.4112508|
 3292| 471| 3.5| 3.4192245|
 3292| 471| 3.5| 3.4192245|
 7004| 471| 3.5| 3.4411166|
 1463| 471| 3.5| 3.4230366|
 3589| 471| 3.5| 3.3911142|
 6624| 471| 3.5| 3.4168422|
 3745| 471| 3.5| 3.4194207|
 967| 471| 3.5| 3.385782|
 5881| 471| 3.5| 3.4110682|
+------+------------+------+----------+
only showing top 20 rows

### Defining a evaluator to check model performance

In [17]:

evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")

In [18]:
RMSE=evaluator.evaluate(test_predictions)

In [19]:
print(RMSE)

0.1248431247447672

### Building the best model using Cross Validation and ParamGridBuilder
### Creating a new training and test data for validation 

In [21]:
(train,test)=restaurant_rating.randomSplit([0.8,0.2],seed=55)

In [22]:
## Creating a base ALS Model
als=ALS(userCol="UserId",itemCol="restaurantId",ratingCol="rating",coldStartStrategy="drop",nonnegative=True,implicitPrefs=False)

###  Adding hyperparameters and their respective values to param_grid

In [23]:

param_grid= ParamGridBuilder() \
            .addGrid(als.rank,[10,50,100]) \
            .addGrid(als.maxIter,[5,15,25]) \
            .addGrid(als.regParam,[.01,.05]) \
            .build()
print ("Num models to be tested: ", len(param_grid))

Num models to be tested: 18

In [24]:
evaluator_cv=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")

### Building cross validation using CrossValidator

In [25]:

cv=CrossValidator(estimator=als,estimatorParamMaps=param_grid,evaluator=evaluator_cv,numFolds=5)

In [26]:
#Fitting cross validator to the 'train' dataset
model = cv.fit(train)

In [27]:
best_model = model.bestModel

In [28]:
best_model.transform(test)

Out[80]: DataFrame[UserId: bigint, restaurantId: bigint, rating: double, prediction: float]

In [29]:
test_predictions = best_model.transform(test)

In [30]:
test_predictions.show()

+------+------------+------+----------+
UserId|restaurantId|rating|prediction|
+------+------------+------+----------+
 4899| 148| 4.0| 3.9488034|
 1247| 148| 4.0| 3.9415193|
 2858| 148| 4.0| 3.9640925|
 905| 148| 4.0| 3.9388468|
 2371| 463| 2.0| 1.9965798|
 1689| 471| 3.5| 3.4413726|
 1774| 471| 3.5| 3.3689234|
 839| 471| 3.5| 3.4632301|
 3135| 471| 3.5| 3.4470525|
 6381| 471| 3.5| 3.4673154|
 4264| 471| 3.5| 3.4383752|
 870| 471| 3.5| 3.455944|
 7004| 471| 3.5| 3.4784327|
 6624| 471| 3.5| 3.4536111|
 2672| 471| 3.5| 3.4573038|
 2601| 496| 4.0| 3.9544122|
 4898| 496| 4.0| 3.936413|
 6508| 496| 4.0| 3.944112|
 871| 496| 4.0| 3.9592502|
 829| 496| 4.0| 3.9427118|
+------+------------+------+----------+
only showing top 20 rows

### Calculate the RMSE of test_predictions

In [31]:

RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

0.09446296264812788

### Reformatting the ALS recommmendation output and generating recommendation for 20 users

In [32]:
ALS_recommendations=model.recommendForAllUsers(20)

In [33]:
ALS_recommendations.registerTempTable("ALS_rcs_temp")

In [34]:
ALS_recommendations.show()

+------+--------------------+
UserId| recommendations|
+------+--------------------+
 1580|[[363, 4.8728747]...|
 4900|[[363, 4.9118967]...|
 5300|[[363, 4.856173],...|
 6620|[[363, 4.7356544]...|
 7240|[[363, 4.9161973]...|
 471|[[363, 4.886008],...|
 1591|[[363, 4.7422147]...|
 4101|[[363, 4.8160486]...|
 1342|[[363, 4.871385],...|
 2122|[[363, 4.790891],...|
 2142|[[363, 4.8255854]...|
 463|[[363, 4.8643823]...|
 833|[[363, 4.870561],...|
 5803|[[363, 4.8861384]...|
 7253|[[363, 4.83219], ...|
 3794|[[363, 4.8913646]...|
 6654|[[363, 4.7556577]...|
 3175|[[363, 4.8573995]...|
 4935|[[363, 4.82674], ...|
 496|[[363, 4.878773],...|
+------+--------------------+
only showing top 20 rows

In [35]:
clean_rcs=spark.sql("SELECT userId,restaurantIds_and_ratings.restaurantId as restaurantId,restaurantIds_and_ratings.rating as prediction from ALS_rcs_temp LATERAL VIEW explode(recommendations) exploded_table as restaurantIds_and_ratings")

In [36]:
clean_rcs.show()

+------+------------+----------+
userId|restaurantId|prediction|
+------+------------+----------+
 1580| 363| 4.8728747|
 1580| 683| 4.8643713|
 1580| 402| 4.8633676|
 1580| 380| 4.8523455|
 1580| 693| 4.814744|
 1580| 587| 4.8080673|
 1580| 5| 4.7921114|
 1580| 55| 4.7100058|
 1580| 262| 4.4284697|
 1580| 549| 4.4238915|
 1580| 565| 4.4109364|
 1580| 454| 4.405081|
 1580| 415| 4.4045997|
 1580| 486| 4.403766|
 1580| 448| 4.403624|
 1580| 35| 4.4027085|
 1580| 717| 4.4021025|
 1580| 146| 4.401228|
 1580| 520| 4.399331|
 1580| 651| 4.39601|
+------+------------+----------+
only showing top 20 rows

In [37]:
clean_rcs.join(restaurant_rating,['restaurantId','userId'],"left").show()

+------------+------+----------+------+
restaurantId|userId|prediction|rating|
+------------+------+----------+------+
 5| 443| 4.7281113| null|
 5| 699| 4.8028235| 5.0|
 5| 815| 4.744085| null|
 5| 866| 4.748084| null|
 5| 928| 4.787408| null|
 5| 975| 4.672315| null|
 5| 1081| 4.7051272| null|
 5| 1565| 4.7361717| null|
 5| 1949| 4.6671205| null|
 5| 2194| 4.786106| null|
 5| 2253| 4.50532| null|
 5| 2410| 4.7512965| null|
 5| 2428| 4.735284| null|
 5| 2557| 4.7861495| null|
 5| 2656| 4.7965875| null|
 5| 3307| 4.670579| null|
 5| 3506| 4.7423477| null|
 5| 3963| 4.797973| null|
 5| 4032| 4.647104| null|
 5| 4173| 4.791643| null|
+------------+------+----------+------+
only showing top 20 rows